In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import time
from pandas import DataFrame
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report, precision_score,accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
standardsc = StandardScaler()
mas = MaxAbsScaler()

In [2]:
tcga = pd.read_csv('TCGA_data.csv')
tcga.drop('cancer', axis = 1, inplace = True)
tcga = tcga.sort_index(axis=1)
tcga['y'] = tcga['y'].replace(['KIRC','KIRP','KICH'],['RC','RC','RC'])

from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
tcga["y_encode"] = lb_make.fit_transform(tcga["y"])

In [3]:
tcga.head()

,cyto.10p11.1,cyto.10p11.21,cyto.10p11.22,cyto.10p11.23,cyto.10p12.1,cyto.10p12.2,cyto.10p12.31,cyto.10p12.32,cyto.10p12.33,cyto.10p13,...,cyto.9q33.1,cyto.9q33.2,cyto.9q33.3,cyto.9q34.11,cyto.9q34.12,cyto.9q34.13,cyto.9q34.2,cyto.9q34.3,y,y_encode
0,-0.1608,0.2213,0.2213,0.2213,0.2213,0.2213,0.2213,0.2213,0.2213,0.2213,...,-0.1732,-0.1732,-0.1732,-0.1732,-0.1732,-0.1732,-0.1732,-0.1732,BLCA,0
1,-0.0017,-0.0017,-0.0017,-0.0017,-0.0017,-0.0017,-0.0017,-0.0017,-0.0017,-0.0017,...,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,Normal,12
2,-0.2733,0.0403,0.0403,0.0403,0.0356,0.0356,0.0356,0.0356,0.0356,0.0356,...,-0.2716,-0.2716,-0.2716,-0.3456,-0.3456,0.2685,-0.3587,-0.3587,BLCA,0
3,0.0013,0.0013,0.0013,0.0013,-0.0057,0.0053,0.0053,0.0053,0.0053,0.0053,...,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,Normal,12
4,-0.0003,-0.0003,-0.0003,-0.0003,-0.0003,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,...,-0.0024,-0.0024,-0.0024,-0.0024,-0.0024,-0.0024,-0.0024,-0.0024,Normal,12


In [5]:
urine = pd.read_csv('urine_k15_0419.csv')
urine['y'] = urine['y'].replace(['NL','RCC_100k'],['Normal','RC'])
urine.drop(['Unnamed: 0','X'],axis=1,inplace=True)
urine = urine.sort_index(axis=1)
urine.y.value_counts()

BLCA      51
Normal    28
PRAD      23
RC        21
Name: y, dtype: int64

In [47]:
urine = pd.read_csv('urine_k15_140cases.csv')
urine['y'] = urine['y'].replace(['BLCA_15k','NL_15k','PRAD_15k','RCC_100k'],['BLCA','Normal','PRAD','RC'])
urine.drop(['Unnamed: 0','X'],axis=1,inplace=True)
urine = urine.sort_index(axis=1)
urine.y.value_counts()

BLCA      42
Normal    28
RC        23
PRAD      23
Name: y, dtype: int64

In [48]:
urine['y_encode'] = urine['y'].replace(['BLCA','Normal','PRAD','RC'],[0,12,16,17])
urine = urine.reset_index(drop = True)
print('Urine data shape:',urine.shape)
urine.y_encode.value_counts()

Urine data shape: (116, 788)


0     42
12    28
17    23
16    23
Name: y_encode, dtype: int64

In [49]:
X = tcga.drop(['y','y_encode'],axis=1)
lst = [i for i in urine.columns if i in X.columns]
print(len(lst))

761


In [50]:
X = tcga.loc[:,lst]
y = tcga['y_encode']
X_urine = urine.loc[:,lst]

standardsc.fit(X)
X_standar = pd.DataFrame(standardsc.transform(X),columns=lst)
urine_standar = pd.DataFrame(standardsc.transform(X_urine),columns=lst)

mas.fit(X)
X_mas = pd.DataFrame(standardsc.transform(X),columns=lst)
urine_mas = pd.DataFrame(mas.transform(X_urine),columns=lst)

In [36]:
# ['BC','NL','PC','RC'],[0,12,16,17]
def pred_accuracy(model,urine_x) :
    pred = model.predict(urine_x)
    df_pred = pd.DataFrame({'actual':urine['y'],'predict':pred})
    
    BC = df_pred.loc[df_pred['actual']==0]
    NL = df_pred.loc[df_pred['actual']==12]
    PC = df_pred.loc[df_pred['actual']==16]
    RC = df_pred.loc[df_pred['actual']==17]
    
    print('BC accuracy')
    print('BC : {0:.4f} NL : {1:.4f} PC : {2:.4f} RC : {3:.4f}'.format((BC['predict']==0).sum() / len(BC),(BC['predict']==12).sum() / len(BC),
                                                                          (BC['predict']==16).sum() / len(BC),(BC['predict']==17).sum() / len(BC)))
    print('\nNL accuracy')
    print('BC : {0:.4f} NL : {1:.4f} PC : {2:.4f} RC : {3:.4f}'.format((NL['predict']==0).sum() / len(NL),(NL['predict']==12).sum() / len(NL),
                                                                      (NL['predict']==16).sum() / len(NL),(NL['predict']==17).sum() / len(NL)))
    print('\nPC accuracy')
    print('BC : {0:.4f} NL : {1:.4f} PC : {2:.4f} RC : {3:.4f}'.format((PC['predict']==0).sum() / len(PC),(PC['predict']==12).sum() / len(PC),
                                                                      (PC['predict']==16).sum() / len(PC),(PC['predict']==17).sum() / len(PC)))
    print('\nRC accuracy')
    print('BC : {0:.4f} NL : {1:.4f} PC : {2:.4f} RC : {3:.4f}'.format((RC['predict']==0).sum() / len(RC),(RC['predict']==12).sum() / len(RC),
                                                                      (RC['predict']==16).sum() / len(RC),(RC['predict']==17).sum() / len(RC)))

# Random Forest

In [42]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 0, n_jobs = -1, max_depth=75,n_estimators=500)
rf.fit(X_mas,y)
pred = rf.predict(urine_mas)
accuracy = (pred==urine.y_encode).sum()/len(pred)
print('accuracy : {0:.4f}'.format(accuracy))

accuracy : 0.0259


array([ 1, 10, 16, 17, 19, 21])

In [51]:
rf = RandomForestClassifier(random_state = 0, n_jobs = -1, max_depth=75,n_estimators=500)
rf.fit(X,y)
pred = rf.predict(X_urine)
accuracy = (pred==urine.y_encode).sum()/len(pred)
print('accuracy : {0:.4f}'.format(accuracy))

accuracy : 0.1897


In [187]:
n = 5
df = pd.DataFrame(rf.predict_proba(urine.loc[:,lst]),columns=lb_make.classes_)
df['class'] = df.apply(lambda x: list(x.sort_values(ascending=False)[:n].index), axis=1)
df['prob'] = df.drop('class',axis=1).apply(lambda x: list(x.sort_values(ascending=False)[:5]), axis=1)

pred_df = pd.DataFrame({'actual':urine.y, 'class':df['class'], 'prob':df['class']})
pred_df.head()

,actual,class,prob
0,BLCA,"[BLCA, BRCA, STAD, LUAD, HNSC]","[BLCA, BRCA, STAD, LUAD, HNSC]"
1,BLCA,"[BLCA, SKCM, PRAD, STAD, BRCA]","[BLCA, SKCM, PRAD, STAD, BRCA]"
2,BLCA,"[Normal, UCEC, LGG, PRAD, THCA]","[Normal, UCEC, LGG, PRAD, THCA]"
3,BLCA,"[Normal, UCEC, BRCA, THCA, PRAD]","[Normal, UCEC, BRCA, THCA, PRAD]"
4,BLCA,"[RC, Normal, BRCA, LGG, LUAD]","[RC, Normal, BRCA, LGG, LUAD]"


In [193]:
def calculate_acc(df) :
    if df['actual'] in df['class'] :
        return 1
    else : return 0

pred_df['accuracy'] = pred_df.apply(calculate_acc, axis=1)

In [199]:
(pred_df['accuracy']==1).sum() / len(pred_df)

0.5086206896551724

## XGBoost

In [62]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state = 0, n_jobs=-1, colsample_bytree= 0.5, max_depth= 5)
xgb.fit(X.as_matrix(),y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [63]:
(xgb.predict(X.as_matrix())==y).sum()/len(y)

0.9716630461018586

In [64]:
accuracy = (xgb.predict(X_urine.as_matrix())==urine.y_encode).sum() / len(urine.y_encode)
print('accuracy : {0:.4f}'.format(accuracy))

accuracy : 0.1466


In [71]:
pred = xgb.predict(X_urine.as_matrix())
df_pred = pd.DataFrame({'actual':urine['y_encode'],'predict':pred})
BC = df_pred.loc[df_pred['actual']==0] # 42
print('BC dataset',BC.shape)
print('accuracy',BC[BC['predict']==0].shape[0]/42)
BC[BC['predict']==0]

BC dataset (42, 2)
accuracy 0.23809523809523808


,actual,predict
0,0,0
8,0,0
9,0,0
10,0,0
12,0,0
15,0,0
17,0,0
26,0,0
27,0,0
28,0,0


In [61]:
xgb = XGBClassifier(random_state = 0, n_jobs=-1, colsample_bytree= 0.5, max_depth= 5)
xgb.fit(X_mas.as_matrix(),y)
accuracy = (xgb.predict(urine_mas.as_matrix())==urine.y_encode).sum() / len(urine.y_encode)
print('accuracy : {0:.4f}'.format(accuracy))

accuracy : 0.1207


# Logistic

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler,MaxAbsScaler
from sklearn.linear_model import LogisticRegression

standardsc = StandardScaler()
mms = MinMaxScaler()
mas = MaxAbsScaler()

lr = LogisticRegression()

In [ ]:
model_pipeline(lr)

0 번째 accuracy non_scale : 0.8048% StandardScale : 0.7896% MinMax : 0.5000% MaxAbs : 0.7944%
1 번째 accuracy non_scale : 0.7990% StandardScale : 0.7821% MinMax : 0.4958% MaxAbs : 0.7886%
2 번째 accuracy non_scale : 0.7999% StandardScale : 0.7801% MinMax : 0.4975% MaxAbs : 0.7929%
3 번째 accuracy non_scale : 0.8035% StandardScale : 0.7786% MinMax : 0.4978% MaxAbs : 0.7876%


In [7]:
from sklearn.linear_model import LogisticRegression
start_time = time.time()
lr = LogisticRegression(n_jobs=-1)
lr.fit(X,y)
print('sec : ',time.time()-start_time) # 2 minutes

sec :  108.8355324268341


In [8]:
pred = lr.predict(urine.drop('y', axis = 1))
accuracy = accuracy_score(urine['y'],pred)
print('Logistic accuracy : {0:.4f}'.format(accuracy))

Logistic accuracy : 0.3253


In [9]:
pred_accuracy(lr)

BC accuracy : 0.1190
Normal accuracy : 1.0000
PC accuracy : 0.0500


In [11]:
pred = lr.predict(urine.drop('y', axis = 1))
df_pred = pd.DataFrame({'actual':urine['y'],'predict':pred})


In [23]:
df_pred[df_pred['actual']==0].predict.value_counts()/len(df_pred[df_pred['actual']==0])

15    0.309524
0     0.119048
5     0.095238
27    0.071429
19    0.071429
13    0.071429
2     0.071429
25    0.047619
11    0.047619
1     0.047619
14    0.023810
10    0.023810
Name: predict, dtype: float64

In [17]:
df_pred[df_pred['actual']==15].predict.value_counts()

15    21
Name: predict, dtype: int64

In [24]:
df_pred[df_pred['actual']==19].predict.value_counts()/len(df_pred[df_pred['actual']==19])

15    0.60
5     0.10
3     0.10
27    0.05
19    0.05
11    0.05
1     0.05
Name: predict, dtype: float64

# LightGBM

In [15]:
from lightgbm import LGBMClassifier
start_time = time.time()

lgbm_clf = LGBMClassifier(n_estimators=200, n_jobs=-1)
lgbm_clf.fit(X, y, eval_metric="logloss", verbose=50)
print('sec : ',time.time()-start_time) # 4 minutes

sec :  236.48080492019653


In [16]:
pred = lgbm_clf.predict(urine.drop('y', axis = 1))
accuracy = accuracy_score(urine['y'], pred)
print('LightGBM accuracy : {0:.4f}'.format(accuracy))

LightGBM accuracy : 0.3373


In [17]:
pred_accuracy(lgbm_clf)

BC accuracy : 0.1667
Normal accuracy : 1.0000
PC accuracy : 0.0000


In [50]:
urine140 = pd.read_csv('urine_k15_0419.csv')
urine120 = pd.read_csv('urine_k15_140cases.csv')

# urine['y'] = urine['y'].replace(['NL','RCC_100k'],['Normal','RC'])
# urine.drop(['Unnamed: 0','X'],axis=1,inplace=True)
# urine = urine.sort_index(axis=1)
# urine.y.value_counts()